In [5]:
import argparse
import mlflow
from fraud_trainer import load_data, train_and_log
from helpers.domino_short_id import domino_short_id
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB


def train_fraud():
    # parser = argparse.ArgumentParser(description="Train fraud classifiers and log to MLflow")
    # parser.add_argument('--clean-filename', required=True, help='CSV file name under data/<project>')
    # parser.add_argument('--model', choices=['xgb','ada','all'], default='all', help='Which model(s) to run')
    # parser.add_argument('--experiment-name', default="CC Fraud Classifier Training", help='MLflow experiment name')
    # parser.add_argument('--random-state', type=int, default=2018)
    # args = parser.parse_args()
    clean_filename = 'cleaned_cc_transactions.csv'
    experiment_name = f"CC Fraud Classifier Training {domino_short_id()}"
    model = 'ada'
    random_state = None
    
    # Set up experiment
    mlflow.set_experiment(experiment_name)

    # Load data once
    df, X_train, X_val, y_train, y_val, features = load_data(
        clean_filename,
        random_state=random_state
    )
    clean_filename = 'cleaned_cc_transactions.csv'
    
    xgb = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric="auc",
        random_state=random_state
    )
    
    ada = AdaBoostClassifier(
        n_estimators=10,
        learning_rate=0.1,
        random_state=random_state
    )
    
    gnb = GaussianNB()

    model_list = [
        (xgb, 'XGBoost'),
        (ada, 'AdaBoost'),
        (gnb, 'NaiveBayes')
    ]
    
    for model, name in model_list:
        print(f'training model {name}')
        train_and_log(
            model, name,
            df, X_train, X_val, y_train, y_val,
            features, clean_filename
        )

for i in range(15):
    train_fraud()


training model XGBoost


Registered model 'CC Fraud XGBoost Classifier' already exists. Creating a new version of this model...
2025/06/30 20:48:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CC Fraud XGBoost Classifier, version 76
Created version '76' of model 'CC Fraud XGBoost Classifier'.


🏃 View run XGBoost at: http://127.0.0.1:8768/#/experiments/1566/runs/177cbdda5cfd4ddca89d85d6d10e8f82
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1566
training model AdaBoost


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
Registered model 'CC Fraud AdaBoost Classifier' already exists. Creating a new version of this model...
2025/06/30 20:48:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CC Fraud AdaBoost Classifier, version 82
Created version '82' of model 'CC Fraud AdaBoost Classifier'.


🏃 View run AdaBoost at: http://127.0.0.1:8768/#/experiments/1566/runs/2936b6f9aaeb45c1bf894e49f979a4ce
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1566
training model NaiveBayes


...

🏃 View run XGBoost at: http://127.0.0.1:8768/#/experiments/1566/runs/dc2517fa2a10497ca1279be59b88d042
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1566
training model AdaBoost


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
Registered model 'CC Fraud AdaBoost Classifier' already exists. Creating a new version of this model...
2025/06/30 20:55:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CC Fraud AdaBoost Classifier, version 96
Created version '96' of model 'CC Fraud AdaBoost Classifier'.


🏃 View run AdaBoost at: http://127.0.0.1:8768/#/experiments/1566/runs/1347017d17bc47e6ba72bcc5b6c1c575
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1566
training model NaiveBayes


Registered model 'CC Fraud NaiveBayes Classifier' already exists. Creating a new version of this model...
2025/06/30 20:55:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CC Fraud NaiveBayes Classifier, version 31
Created version '31' of model 'CC Fraud NaiveBayes Classifier'.


🏃 View run NaiveBayes at: http://127.0.0.1:8768/#/experiments/1566/runs/6788b2e08b4542318eb30bf702837e2e
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1566
